# Road segmentation

In this notebook, we'll be segmenting road networks from aerial imagery.

In [ ]:
import rioxarray

## Get image data from OpenAerialMap

- OpenAerialMap images over Vanuatu - https://map.openaerialmap.org/#/168.3819580078125,-16.688816956180833,7

We'll be using a Maxar Worldview-2 image with a spatial resolution of 32cm over Port Vila on 2024 October 19.

- Preview at https://map.openaerialmap.org/#/168.31419467926025,-17.73086527059167,14/square/311123113/676737799f511a0001cc98c1

In [ ]:
image_url = "https://oin-hotosm-temp.s3.us-east-1.amazonaws.com/676733089f511a0001cc98b6/0/676733089f511a0001cc98b7.tif"

The RGB images are distributed in a Cloud-optimized GeoTIFF (COG) format.
We'll follow https://corteva.github.io/rioxarray/stable/examples/COG.html to open the file in Python.

**Note**: We set `overview_level=3` to get a lower resolution image.
The spatial resolutions at different overview levels are:
- Level -1 (native): 0.32 meters
- Level 0: 0.64 meters
- Level 1: 1.28 meters
- Level 2: 2.57 meters
- Level 3: 5.14 meters

In [ ]:
rds = rioxarray.open_rasterio(filename=image_url, overview_level=3)
rds

In [ ]:
# Check spatial resolution in meters
rds.rio.resolution()

In [ ]:
# Check coordinate reference system
rds.rio.crs.to_string()

In [ ]:
# Check bounding box extent
bbox = rds.rio.bounds()
bbox

The image in an `xarray.DataArray` can be plotted using `.plot.imshow(rgb="band")`

In [ ]:
rds.plot.imshow(rgb="band")

There are some black NoData/NaN areas, let's crop them out using
[`.rio.clip_box`](https://corteva.github.io/rioxarray/stable/examples/clip_box.html#Clip-using-a-bounding-box).

In [ ]:
rds_portvila = rds.rio.clip_box(minx=18732000, miny=-2012000, maxx=18742000, maxy=-2002000)

In [ ]:
rds_portvila.plot.imshow(rgb="band")